In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gender-submisson/All 1.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
# split a dataset into training and testing sets
from sklearn.model_selection import train_test_split
# scales your values to be between 0 and 1,encode categorical labels into numerical format
from sklearn.preprocessing import StandardScaler, LabelEncoder
# handling missing values
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
# evaluating the performance 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
# Load datasets
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Combine train and test datasets for preprocessing
combined_data = pd.concat([train_data, test_data])
combined_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
combined_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


As we can see Survived,Age,Fare and embarked have missing values which have to be dealt with and the categorical variables have to be changed to numerical

In [9]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
combined_data['Age'] = imputer.fit_transform(combined_data[['Age']])
combined_data['Fare'] = imputer.fit_transform(combined_data[['Fare']])
combined_data['Survived'] = imputer.fit_transform(combined_data[['Survived']])

In [10]:
combined_data['Embarked'] = combined_data['Embarked'].fillna(combined_data['Embarked'].mode().iloc[0])

In [11]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1309 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [12]:
# Encode categorical variables
label_encoder = LabelEncoder()
combined_data['Sex'] = label_encoder.fit_transform(combined_data['Sex'])
combined_data['Embarked'] = label_encoder.fit_transform(combined_data['Embarked'])

In [13]:
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
gender_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


Now that we have already cleaned and encoded our data its time to split it back to train and test set

In [15]:
# Select your dependant and independant variables
X = combined_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
y = combined_data['Survived']

In [16]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,22.000000,1,0,7.2500
1,1,0,38.000000,1,0,71.2833
2,3,0,26.000000,0,0,7.9250
3,1,0,35.000000,1,0,53.1000
4,3,1,35.000000,0,0,8.0500
...,...,...,...,...,...,...
413,3,1,29.881138,0,0,8.0500
414,1,0,39.000000,0,0,108.9000
415,3,1,38.500000,0,0,7.2500
416,3,1,29.881138,0,0,8.0500


In [17]:
y

0      0.000000
1      1.000000
2      1.000000
3      1.000000
4      0.000000
         ...   
413    0.383838
414    0.383838
415    0.383838
416    0.383838
417    0.383838
Name: Survived, Length: 1309, dtype: float64

In [18]:
# Split the combined dataset back into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [19]:
y_train = label_encoder.fit_transform(y_train)

In [20]:
y_test = label_encoder.fit_transform(y_test)

In [21]:
# Initialize the Random Forest classifier
clf = RandomForestClassifier(random_state=43)

In [22]:
# Train the classifier
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=43)

In [23]:
# Evaluate the model
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [25]:
# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.5

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.68      0.63       110
           1       0.31      0.24      0.27        87
           2       0.52      0.54      0.53        65

    accuracy                           0.50       262
   macro avg       0.47      0.49      0.48       262
weighted avg       0.48      0.50      0.49       262



In [27]:
print(confusion_matrix(y_test, y_pred))

[[75 25 10]
 [44 21 22]
 [ 9 21 35]]


In [28]:
# make predictions on the test set
# Preprocess the test set
test_features = combined_data.iloc[len(train_data):][['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
test_features['Sex'] = label_encoder.transform(test_features['Sex'])

In [29]:
# Make predictions on the test set
test_predictions = clf.predict(test_features)
test_predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 2,
       1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 2, 1,
       2, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 1, 2, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 1, 1,

In [30]:
# Create a submission DataFrame using passenger IDs from gender_submission and predicted survival values
submission_df = pd.DataFrame({
    'PassengerId': gender_submission['PassengerId'],
    'Survived': test_predictions})

In [31]:
submission_df

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,1
416,1308,0


In [35]:
# Save the submission DataFrame to a CSV file
submission_df.to_csv('titanic_submission.csv', index=False)